> **Rappel** : clique sur une cellule grise, puis **Shift + Entree** pour l'executer.
> Execute les cellules **dans l'ordre** de haut en bas.

---

# Lecon 6 : Entrainer le modele

## Le moment de verite !

Dans la lecon 5, on a construit un mini-LLM complet : embeddings,
attention, MLP, softmax. Mais ses poids etaient **aleatoires** --
il ne savait rien et generait du charabia.

Aujourd'hui, on va lui **apprendre** a generer des noms de Pokemon.

C'est comme un joueur qui decouvre des centaines de Pokemon et finit
par comprendre "comment ca sonne", un nom de Pokemon.

> L'entrainement va prendre quelques minutes. Pendant que le modele
> apprend, tu pourras lire les sections "En vrai..." plus bas !

In [ ]:
from IPython.display import HTML, display

_exercices_faits = set()
_NB_TOTAL = 3


def verifier(num_exercice, condition, message_ok, message_aide=""):
    """Valide un exercice avec feedback HTML vert/rouge + compteur."""
    if condition:
        _exercices_faits.add(num_exercice)
        n = len(_exercices_faits)
        barre = "\U0001f7e9" * n + "\u2b1c" * (_NB_TOTAL - n)
        display(
            HTML(
                f'<div style="padding:10px;background:#d4edda;border-left:4px solid #28a745;'
                f'margin:8px 0;border-radius:4px;font-family:sans-serif">'
                f"\u2705 <b>{message_ok}</b><br>"
                f'<span style="color:#555">Progression : {barre} {n}/{_NB_TOTAL}</span></div>'
            )
        )
        if n == _NB_TOTAL:
            display(
                HTML(
                    '<div style="padding:12px;background:linear-gradient(135deg,#667eea,#764ba2);'
                    "color:white;border-radius:8px;text-align:center;font-family:sans-serif;"
                    'font-size:1.2em;margin:8px 0">\U0001f3c6 <b>Bravo ! Toutes les activites de cette lecon sont terminees !</b></div>'
                )
            )
    else:
        display(
            HTML(
                f'<div style="padding:10px;background:#fff3cd;border-left:4px solid #ffc107;'
                f'margin:8px 0;border-radius:4px;font-family:sans-serif">'
                f"\U0001f4a1 <b>{message_aide}</b></div>"
            )
        )


def afficher_evolution_loss(pertes, titre="Courbe de loss"):
    """Affiche la loss sous forme de barres verticales HTML avec degrade."""
    if not pertes:
        return
    max_loss = max(pertes)
    min_loss = min(pertes)
    bars = ""
    n = len(pertes)
    bar_w = max(4, min(40, 600 // n))
    for i, loss in enumerate(pertes):
        h = int((loss / max_loss) * 120) if max_loss > 0 else 0
        ratio = (loss - min_loss) / (max_loss - min_loss) if max_loss > min_loss else 0
        r = int(220 * ratio + 30)
        g = int(180 * (1 - ratio) + 50)
        bars += (
            f'<div style="display:inline-block;width:{bar_w}px;vertical-align:bottom;'
            f'margin:0 1px;height:{h}px;background:rgb({r},{g},60);border-radius:2px 2px 0 0" '
            f'title="Epoch {i + 1}: {loss:.3f}"></div>'
        )
    display(
        HTML(
            f'<!-- tuto-viz --><div style="margin:8px 0"><b>{titre}</b>'
            f'<div style="display:flex;align-items:flex-end;height:140px;padding:8px;'
            f'background:#f8f9fa;border-radius:4px;margin-top:4px">{bars}</div>'
            f'<div style="display:flex;justify-content:space-between;color:#555;font-size:0.8em;margin-top:2px">'
            f"<span>Epoch 1 (loss={pertes[0]:.2f})</span><span>Epoch {len(pertes)} (loss={pertes[-1]:.2f})</span></div></div>"
        )
    )


def afficher_barres(valeurs, etiquettes, titre="Probabilites"):
    """Affiche des barres horizontales HTML."""
    rows = ""
    max_val = max(valeurs) if valeurs else 1
    for etiq, val in zip(etiquettes, valeurs, strict=False):
        pct = val / max_val * 100 if max_val > 0 else 0
        rows += (
            f'<tr><td style="padding:3px 8px;font-weight:bold;font-size:1em">{etiq}</td>'
            f'<td style="padding:3px;width:300px"><div style="background:linear-gradient(90deg,#667eea,#764ba2);'
            f'width:{max(pct, 2):.0f}%;height:20px;border-radius:4px"></div></td>'
            f'<td style="padding:3px 8px;font-size:0.9em">{val:.1%}</td></tr>'
        )
    display(
        HTML(
            f'<!-- tuto-viz --><div style="margin:8px 0"><b>{titre}</b>'
            f'<table style="border-collapse:collapse;margin-top:4px">{rows}</table></div>'
        )
    )


print("Outils de visualisation charges !")

---
## Etape 1 : Charger les noms de Pokemon

Dans les lecons precedentes, on utilisait une poignee de Pokemon
ecrits a la main. Maintenant, on utilise un vrai dataset :

**~1 000 noms de Pokemon** tires de la PokeAPI ((c) Nintendo).

In [ ]:
import math
import random
import time

random.seed(42)

# Noms de Pokemon (c) Nintendo / Creatures Inc. / GAME FREAK inc.
# Source : PokeAPI (https://pokeapi.co/) -- usage educatif uniquement
_POKEMON_DATA = """
abo
abra
absol
aeromite
aeropteryx
aflamanoir
airmure
akwakwak
alakazam
aligatueur
altaria
amaama
amagara
amassel
amonistar
amonita
amovenus
amphinobi
ampibidou
anchwatt
angoliath
anorith
apireine
apitrini
aquali
arakdo
araqua
arbok
arboliva
arcanin
arceus
archeduc
archeodong
archeomire
arcko
argouste
arkeapti
armaldo
armulys
arrozard
artikodin
aspicot
astronelle
avaltout
axoloto
azumarill
azurill
babimanta
bacabouh
badabouin
baggaid
baggiguane
balbaleze
balbuto
balignon
bamboiselle
banshitrouye
baojian
barbicha
bargantua
barloche
barpau
bastiodon
batracne
baudrive
bazoucan
bebecaille
bekaglacon
bekipan
beldeneige
berasca
berserkatt
betochef
bibichut
blancoton
bleuseille
blindalys
blindepique
blizzaroi
blizzeval
blizzi
boguerisse
bombydou
boreas
boskara
bouldeneu
boumata
bourrinos
boustiflor
braisillon
branette
brasegali
brindibou
briochien
brocelome
brouhabam
brutalibre
brutapode
bruyverne
bulbizarre
cablifere
cabriolaine
cacnea
cacturne
cadoizo
camerupt
canarbello
canarticho
cancrelove
candine
caninos
capidextre
capumain
carabaffe
carabing
carapagos
carapuce
caratroc
carchacrok
carmache
carmadura
carvanha
castorno
celebi
cerbyllin
cerfrousse
ceribou
ceriflor
chacripan
chaffreux
chaglam
chamallot
chapignon
chapotus
charbambin
charbi
charibari
charkos
charmillon
charmilly
charmina
charpenti
chartor
chefdefer
chelours
chenipan
chenipotte
cheniselle
cheniti
chetiflor
chevroum
chimpenfeu
chinchidou
chlorobule
chochodile
chongjian
chovsourir
chrysacier
chrysapile
chuchmur
cizayox
clamiral
cleopsytra
clic
cliticlic
coatox
cobaltium
cochignon
coconfort
cocotine
coiffeton
coleodome
colhomard
colimucus
colombeau
colossinge
compagnol
concombaffe
coquiperl
corayome
corayon
corboss
cornebre
corvaillus
cosmog
cosmovum
cotovol
couafarel
couaneton
coudlangue
coupenotte
courrousinge
couverdure
coxy
coxyclaque
crabagarre
crabaraque
crabicoque
crabominable
cradopaud
craparoi
crapustule
crefadet
crefollet
crehelf
cremy
cresselia
crikzik
croaporal
crocogril
crocorible
crocrodil
croquine
crustabri
cryodo
cryptero
cupcanaille
dardargnan
darkrai
darumacho
darumarond
debugant
dedenne
deflaisan
delcatty
delestin
demanta
demeteros
demolosse
denticrisse
deoxys
desseliande
deusolourdo
dialga
diamat
diancie
dimocles
dimoret
dinglu
dinoclier
dispareptil
dodoala
dodrio
doduo
dofin
dogrino
dolman
donphan
doudouvet
draby
dracaufeu
drackhaus
draco
dracolosse
dragmara
draieul
drakkarmin
drascore
dratatin
drattak
dunaconda
dunaja
duralugon
dynavolt
ecaid
ecayon
ecrapince
ecremeuh
ectoplasma
effleche
ekaiser
elecsprint
electhor
electrode
elekable
elekid
elektek
embrochet
embrylex
emolga
empiflor
engloutyran
entei
eoko
epinedefer
escargaume
escroco
ethernatos
etouraptor
etourmi
etourvol
evoli
exagide
excavarenne
excelangue
famignol
fantominus
fantyrm
farfaduvet
farfuret
farfurex
farigiraf
favianos
felicanis
felinferno
ferdeter
fermite
ferosinge
feuforeve
feuillajou
feuiloutan
feunard
feunnec
feupercant
feurisson
filentrappe
flabebe
flagadoss
flamajou
flambino
flambusard
flamenroule
flamiaou
flamigator
flamoutan
flingouste
flobio
floette
floramantis
floravol
floreclat
florges
florizarre
flotajou
flotillon
flotoutan
flottemeche
fluvetin
fongusfurie
foretress
forgelina
forgella
forgerette
fortivoire
fortusimia
fouinar
fouinette
fourbelin
fragilady
fragroin
frigodo
frison
frissonille
froussardine
fulgudog
fulgulairo
fulguris
funecire
furaiglon
galegon
galekid
galeking
galifeu
gallame
galopa
galvagla
galvagon
galvaran
gambex
gamblast
gardedefer
gardevoir
gaulet
genesect
geolithe
germeclat
germignon
gigalithe
gigansel
girafarig
giratina
givrali
glaivodo
gloupti
gobou
goelise
goinfrex
golemastoc
golgopathe
gorythmic
goupelin
goupilou
goupix
gourmelet
gouroutan
grahyena
grainipiot
granbull
granivol
gravalanch
grelacon
grenousse
gribouraigne
griknot
grillepattes
grimalin
grindur
gringolem
grodoudou
grodrive
grolem
gromago
grondogue
groret
grotadmorv
grotichon
groudon
gruikui
gueriaigle
guerilande
hachecateur
hariyama
hastacuda
haydaim
heatran
heledelle
heliatronc
helionceau
herbizarre
hericendre
hexadron
hexagel
hippodocus
hippopotas
hooh
hoopa
hoothoot
hottedefer
hurlequeue
hydragla
hydragon
hypnomade
hypocean
hyporoi
hypotrempe
iguolta
incisache
insecateur
insolourdo
irefoudre
ixon
jirachi
joliflor
judokrak
jungko
kabuto
kabutops
kadabra
kaiminus
kaimorse
kangourex
kaorine
kapoera
karaclee
katagami
kecleon
keldeo
keunotor
khelocrok
kicklee
kirlia
kokiyas
koraidon
korillon
krabboss
krabby
kraknoix
krakos
kranidos
kravarech
kungfouine
kyogre
kyurem
laggron
lainergie
lakmecygne
lamantine
lamperoie
lampignon
lancargot
lanssorien
lanturn
laporeille
lapyro
larmeleon
larvadar
larveyette
larvibule
latias
latios
leboulerou
leopardus
lepidonille
lestombaile
leuphorie
leveinard
leviator
lewsor
lezargus
lianaja
libegon
lilia
lilliterelle
limagma
limaspeed
limonde
lineon
lippouti
lippoutou
lixy
lockpin
lokhlass
lombre
lougaroc
loupio
lovdisc
lucanon
lucario
ludicolo
lugia
lugulabre
lumineon
lumivole
lunala
luxio
luxray
machoc
machopeur
mackogneur
macronium
maganon
magby
magearna
magicarpe
magireve
magmar
magneti
magneton
magnezone
majaspic
makuhita
malamandre
malosse
malvalame
mamanbo
mammochon
manaphy
mandrillon
manglouton
mangriff
manternel
manzai
maracachi
maraiste
marcacrin
marill
marisson
marshadow
mascaiman
maskadra
massko
mastouffe
mateloutre
matoufeu
matourgeon
medhyena
meditikka
meganium
megapagos
meios
melancolux
melmetal
melo
melodelfe
meloetta
melofee
melokrik
meltan
mentali
mesmerella
metalosse
metamorph
metang
meteno
mew
mewtwo
mglaquette
miamiasme
miaouss
miascarade
miasmax
migalos
milobellus
mimantis
mimejr
mimigal
mimiqui
mimitoss
minidraco
minisange
minotaupe
miradar
miraidon
mistigrix
mitedefer
mmime
momartik
monaflemit
monorpale
monthracite
mordudor
morpeko
morpheo
motisma
motorizard
moufflair
moufouette
moumouflon
moumouton
mouscoto
moustillon
moyade
muciole
mucuscule
munja
munna
muplodocus
mushana
mustebouee
musteflott
mygavolt
mysdibule
mystherbe
nanmeouie
natu
necrozma
negapi
neitram
nemelios
nenupiot
nidoking
nidoqueen
nidoran
nidorina
nidorino
nigirigon
nigosier
ningale
ninjask
nirondelle
noacier
noadkoko
noarfang
noctali
noctunoir
nodulithe
noeunoeuf
nosferalto
nosferapti
nostenfer
nounourson
nucleos
nymphali
obalie
octillery
ogerpon
ohmassacre
okeoke
olivado
olivini
oniglali
onix
opermine
oratoria
ortide
ossatueur
osselait
otaquin
otaria
otarlette
ouistempo
ouisticram
ouvrifier
oyacata
pachirisu
pachyradjah
palarticho
palkia
palmaval
pandarbare
pandespiegle
papilord
papilusion
papinox
paragruel
paras
parasect
parecool
pashmilla
passerouge
patachiot
paumedefer
pechaminus
pelagesable
peregrain
persian
phanpy
pharamp
phione
phogleur
phyllali
piafabec
picassaut
pichu
piclairon
pierroteknik
pietace
pifeuil
pijako
pikachu
pimito
pingoleon
pitrouille
plumeline
pohm
pohmarmotte
pohmotte
poichigeon
poissirene
poissoroy
polagriffe
polarhume
polichombr
poltchageist
polthegeist
pomdepik
pomdorochi
pomdramour
pomdrapi
ponchien
ponchiot
pondralugon
ponyta
porygon
porygonz
posipi
poulpaf
poussacha
poussifeu
predasterie
prinplouf
prismillon
psykokwak
psystigri
ptera
ptiravi
ptitard
ptyranidur
pyrax
pyrobut
pyroli
pyronille
quartermac
queulorior
qulbutoke
qwilfish
qwilpik
racaillou
rafflesia
raichu
raikou
ramboum
ramoloss
rampeailes
rapasdepic
rapion
ratentif
rattata
rattatac
rayquaza
regice
regidrago
regieleki
regigigas
regirock
registeel
relicanth
remoraid
reptincel
reshiram
rexillius
rhinastoc
rhinocorne
rhinoferos
rhinolove
riolu
rocabot
rocdefer
roigada
roitiflam
rondoudou
ronflex
rongourmand
rongrigou
rosabyss
roselia
roserade
rototaupe
roublenard
roucarnage
roucool
roucoups
rouedefer
roussil
rozbouton
rubombelle
rugitlune
sabelette
sablaireau
salameche
salarsen
sancoki
sapereau
saquedeneu
sarmurai
scalpereur
scalpion
scalproie
scarabrute
scarhino
scobolide
scolocendre
scorplane
scorvol
scovilain
scrutella
seleroc
selutin
sepiatop
sepiatroce
seracrawl
serpang
serpenteeau
seviper
shaofouine
sharpedo
shaymin
shifours
siderella
silvallie
simiabraz
simularbre
sinistrail
skelenox
skitty
smogo
smogogo
snubbull
solaroc
solgaleo
solochi
sonistrelle
soporifik
sorbebe
sorbouboul
sorboul
sorcilence
sovkipou
spectreval
spectrum
spinda
spiritomb
spododo
spoink
stalgamin
stari
staross
statitik
steelix
strassie
sucreine
sucroquin
suicune
sulfura
superdofin
sylveroy
symbios
tadmorv
tagtag
tapatoes
tarenbulle
tarinor
tarinorme
tarpaud
tarsal
tartard
taupikeau
taupiqueur
tauros
teddiursa
tenefix
tengalice
tentacool
tentacruel
teraclope
terapagos
terhal
terracool
terracruel
terraiste
terrakium
tetampoule
tetarte
tetesdefer
theffroi
theffroyable
tiboudet
tic
tiplouf
tissenboule
togedemaru
togekiss
togepi
togetic
tokopisco
tokopiyon
tokorico
tokotoro
tomberro
torgamord
tortank
torterra
tortipouss
toudoudou
tournegrin
tournicoton
toutombe
toxizap
tranchodon
trepassable
triopikeau
triopikeur
trioxhydre
tritonde
tritosor
tritox
trompignon
tropius
trousselin
tutafeh
tutankafer
tutetekri
tygnon
tylton
type
typhlosion
tyranocif
ursaking
ursaring
vacilys
vaututrice
vemini
venalgue
venipatte
verpom
vertdefer
vibraninf
victini
vigoroth
vipelierre
virevorreur
viridium
virovent
viskuse
vivaldaim
volcanion
volcaropod
voltali
voltorbe
voltoutou
vorasterie
vortente
vostourno
vrombi
vrombotor
wagomine
wailmer
wailord
wattapik
wattouat
wimessir
wushours
xatu
xerneas
yanma
yanmega
ymphect
yuyu
yveltal
zacian
zamazenta
zapetrel
zarbi
zarude
zebibron
zeblitz
zekrom
zeraora
zeroid
zigzaton
zoroark
zorua
zygarde
"""

pokemons = [nom for nom in _POKEMON_DATA.strip().split("\n") if nom.strip()]

# Quelques stats
longueurs = [len(p) for p in pokemons]
print(f"Pokemon disponibles : {len(pokemons)}")
print(f"Longueur moyenne : {sum(longueurs) / len(longueurs):.1f} lettres")
print(f"Plus court : {min(longueurs)} lettres, plus long : {max(longueurs)} lettres")
print(f"Exemples : {', '.join(pokemons[:8])}")

---
## Etape 2 : Preparer le modele

On reprend **exactement la meme architecture** que la lecon 5
(embeddings + attention + MLP) avec les memes dimensions :

| Parametre | Valeur | Rappel |
|-----------|--------|--------|
| Dimension embeddings | 16 | Chaque lettre = 16 nombres |
| Taille MLP | 32 | Reseau de neurones interne |
| Contexte | 8 | Fenetre de 8 lettres max |
| Parametres | ~2 800 | Les nombres que le modele va ajuster |

C'est exactement le modele de la lecon 5, mais cette fois on va
l'entrainer **pour de vrai** !

In [ ]:
# --- Vocabulaire ---
VOCAB = list(".abcdefghijklmnopqrstuvwxyz")
VOCAB_SIZE = len(VOCAB)  # 27
char_to_id = {c: i for i, c in enumerate(VOCAB)}
id_to_char = {i: c for i, c in enumerate(VOCAB)}

# --- Configuration (memes dimensions que lecon 5) ---
EMBED_DIM = 16
CONTEXT = 8
HIDDEN_DIM = 32

nb_params = (
    VOCAB_SIZE * EMBED_DIM  # tok_emb
    + CONTEXT * EMBED_DIM  # pos_emb
    + 3 * EMBED_DIM * EMBED_DIM  # Wq, Wk, Wv
    + HIDDEN_DIM * EMBED_DIM  # W1
    + HIDDEN_DIM  # b1
    + EMBED_DIM * HIDDEN_DIM  # W2
    + EMBED_DIM  # b2
    + VOCAB_SIZE * EMBED_DIM  # W_out
)

# --- Fonctions utilitaires (memes que lecon 5) ---


def rand_matrix(rows, cols, scale=0.3):
    return [[random.gauss(0, scale) for _ in range(cols)] for _ in range(rows)]


def rand_vector(size, scale=0.3):
    return [random.gauss(0, scale) for _ in range(size)]


def softmax(scores):
    max_s = max(scores)
    exps = [math.exp(s - max_s) for s in scores]
    total = sum(exps)
    return [e / total for e in exps]


def mat_vec(mat, vec):
    return [sum(mat[i][j] * vec[j] for j in range(len(vec))) for i in range(len(mat))]


# --- Initialisation des poids (aleatoires) ---
tok_emb = rand_matrix(VOCAB_SIZE, EMBED_DIM, 0.5)
pos_emb = rand_matrix(CONTEXT, EMBED_DIM, 0.5)
Wq = rand_matrix(EMBED_DIM, EMBED_DIM, 0.2)
Wk = rand_matrix(EMBED_DIM, EMBED_DIM, 0.2)
Wv = rand_matrix(EMBED_DIM, EMBED_DIM, 0.2)
W1 = rand_matrix(HIDDEN_DIM, EMBED_DIM, 0.2)
b1 = rand_vector(HIDDEN_DIM, 0.1)
W2 = rand_matrix(EMBED_DIM, HIDDEN_DIM, 0.2)
b2 = rand_vector(EMBED_DIM, 0.1)
W_out = rand_matrix(VOCAB_SIZE, EMBED_DIM, 0.2)

print(f"Mini-LLM initialise avec {nb_params} parametres aleatoires.")

In [ ]:
def forward_avec_cache(sequence_ids):
    """Forward pass qui sauvegarde les etapes pour le backward."""
    n = len(sequence_ids)

    # 1. Embeddings : token + position
    hidden = []
    for i, tok_id in enumerate(sequence_ids):
        h = [tok_emb[tok_id][d] + pos_emb[i % CONTEXT][d] for d in range(EMBED_DIM)]
        hidden.append(h)

    # 2. Self-Attention (derniere position uniquement)
    q = mat_vec(Wq, hidden[-1])

    scores_bruts = []
    cles = []
    valeurs = []
    for i in range(n):
        k = mat_vec(Wk, hidden[i])
        v = mat_vec(Wv, hidden[i])
        score = sum(q[d] * k[d] for d in range(EMBED_DIM)) / math.sqrt(EMBED_DIM)
        scores_bruts.append(score)
        cles.append(k)
        valeurs.append(v)

    poids_attn = softmax(scores_bruts)

    sortie_attn = [0.0] * EMBED_DIM
    for i in range(n):
        for d in range(EMBED_DIM):
            sortie_attn[d] += poids_attn[i] * valeurs[i][d]

    # Connexion residuelle 1
    x = [hidden[-1][d] + sortie_attn[d] for d in range(EMBED_DIM)]
    x_apres_attn = list(x)

    # 3. MLP
    h1_pre = [
        sum(W1[j][d] * x[d] for d in range(EMBED_DIM)) + b1[j]
        for j in range(HIDDEN_DIM)
    ]
    h1 = [max(0.0, v) for v in h1_pre]  # ReLU
    sortie_mlp = [
        sum(W2[d][j] * h1[j] for j in range(HIDDEN_DIM)) + b2[d]
        for d in range(EMBED_DIM)
    ]

    # Connexion residuelle 2
    x_final = [x[d] + sortie_mlp[d] for d in range(EMBED_DIM)]

    # 4. Sortie
    logits = [
        sum(W_out[v][d] * x_final[d] for d in range(EMBED_DIM))
        for v in range(VOCAB_SIZE)
    ]
    probas = softmax(logits)

    cache = {
        "ids": sequence_ids,
        "hidden": hidden,
        "q": q,
        "cles": cles,
        "valeurs": valeurs,
        "scores_bruts": scores_bruts,
        "poids_attn": poids_attn,
        "sortie_attn": sortie_attn,
        "x_apres_attn": x_apres_attn,
        "h1_pre": h1_pre,
        "h1": h1,
        "sortie_mlp": sortie_mlp,
        "x_final": x_final,
    }
    return probas, cache


# Testons : loss initiale (poids aleatoires)
loss_totale = 0
nb = 0
for pokemon in pokemons[:100]:  # 100 Pokemon pour aller vite
    mot = "." + pokemon + "."
    ids = [char_to_id[c] for c in mot]
    for i in range(1, len(ids)):
        seq = ids[:i][-CONTEXT:]
        cible = ids[i]
        probas, _ = forward_avec_cache(seq)
        loss_totale += -math.log(probas[cible] + 1e-10)
        nb += 1

loss_initiale = loss_totale / nb
print(f"Loss initiale (poids aleatoires) : {loss_initiale:.3f}")
print(f"Loss theorique d'un modele aleatoire : {math.log(VOCAB_SIZE):.3f}")
print(f"  -> Le modele devine au hasard parmi {VOCAB_SIZE} lettres.")

In [ ]:
def generer(debut=".", temperature=0.8, max_len=15):
    """Genere un nom de Pokemon lettre par lettre."""
    ids = [char_to_id[c] for c in debut]
    resultat = debut
    for _ in range(max_len):
        probas, _ = forward_avec_cache(ids[-CONTEXT:])
        if temperature != 1.0:
            logits_t = [math.log(p + 1e-10) / temperature for p in probas]
            probas = softmax(logits_t)
        idx = random.choices(range(VOCAB_SIZE), weights=probas, k=1)[0]
        if idx == char_to_id["."]:
            break
        ids.append(idx)
        resultat += id_to_char[idx]
    return resultat[1:] if resultat.startswith(".") else resultat


print("=== AVANT entrainement (poids aleatoires) ===")
print()
noms_avant = []
for _ in range(10):
    nom = generer()
    noms_avant.append(nom)
    print(f"  {nom.capitalize()}")
print()
print("C'est du charabia ! Le modele ne sait pas ce qu'est un Pokemon.")

---
### A toi de jouer ! (Exercice 1)

Dans la cellule ci-dessous, change `ma_temperature` pour generer
des noms **avant** l'entrainement. Essaie `0.1` (tres sage) ou `2.0` (tres fou).
Tu verras que c'est du charabia dans tous les cas !

In [ ]:
# --- EXERCICE 1 : Change la temperature, puis Shift + Entree ---
ma_temperature = 0.8  # <-- Essaie 0.1 (sage) ou 2.0 (fou) !

print(f"Generation AVANT entrainement (temperature = {ma_temperature}) :")
print()
for _ in range(10):
    nom = generer(temperature=ma_temperature)
    print(f"  {nom.capitalize()}")
print()
print("C'est du charabia ! Sans entrainement, la temperature ne change rien.")

# Validation exercice 1
verifier(
    1,
    ma_temperature != 0.8,
    f"Bien joue ! Avec temperature={ma_temperature}, les noms sont {'sages' if ma_temperature < 0.5 else 'fous' if ma_temperature > 1.5 else 'equilibres'}.",
    "Change ma_temperature pour une autre valeur, par exemple 0.1 ou 2.0.",
)

---
## Etape 3 : La retropropagation

Dans les lecons 2 et 3, on calculait les gradients facilement parce que
le modele etait simple. Maintenant, notre LLM a **7 couches de calcul** :

```
emb -> attention -> residuel -> MLP -> residuel -> W_out -> softmax
```

Pour calculer les gradients, on fait le **chemin inverse** :

```
softmax -> W_out -> residuel -> MLP -> residuel -> attention -> emb
```

C'est la **retropropagation** (backpropagation). L'idee :

1. On part de l'erreur a la sortie (la loss)
2. On remonte couche par couche
3. A chaque couche, on calcule "de combien ce poids a contribue a l'erreur"
4. On ajuste chaque poids dans la bonne direction

> **Analogie** : Imagine une chaine de dominos. Le dernier domino (la loss)
> est tombe trop a droite. Tu remontes la chaine : quel domino a pousse
> trop fort ? C'est celui-la qu'on ajuste.

La formule magique de la sortie est **exactement la meme** que dans les
lecons 2 et 3 :

```
gradient[lettre] = proba_predite[lettre] - (1 si c'est la bonne reponse, 0 sinon)
```

Ensuite, ce gradient se propage en arriere a travers chaque couche.

In [ ]:
def backward(cache, probas, cible):
    """Calcule les gradients -- le chemin inverse du forward."""
    hidden = cache["hidden"]
    q = cache["q"]
    cles = cache["cles"]
    valeurs = cache["valeurs"]
    poids_attn = cache["poids_attn"]
    x_apres_attn = cache["x_apres_attn"]
    h1_pre = cache["h1_pre"]
    h1 = cache["h1"]
    x_final = cache["x_final"]
    ids = cache["ids"]
    n = len(ids)

    # === ETAPE 1 : gradient de la sortie (cross-entropy + softmax) ===
    # Meme formule que lecons 2 et 3 : augmente la bonne reponse, baisse les autres
    d_logits = [probas[v] - (1.0 if v == cible else 0.0) for v in range(VOCAB_SIZE)]

    # === ETAPE 2 : gradient de W_out ===
    d_W_out = [
        [d_logits[v] * x_final[d] for d in range(EMBED_DIM)] for v in range(VOCAB_SIZE)
    ]
    d_x = [
        sum(d_logits[v] * W_out[v][d] for v in range(VOCAB_SIZE))
        for d in range(EMBED_DIM)
    ]

    # === ETAPE 3 : connexion residuelle 2 -> gradient passe aux deux branches ===
    d_mlp = list(d_x)
    d_xa = list(d_x)

    # === ETAPE 4 : backward du MLP ===
    d_W2 = [[d_mlp[d] * h1[j] for j in range(HIDDEN_DIM)] for d in range(EMBED_DIM)]
    d_b2 = list(d_mlp)
    d_h1 = [
        sum(d_mlp[d] * W2[d][j] for d in range(EMBED_DIM)) for j in range(HIDDEN_DIM)
    ]
    # ReLU backward : le gradient passe si h1_pre > 0, sinon bloque
    d_h1p = [d_h1[j] * (1.0 if h1_pre[j] > 0 else 0.0) for j in range(HIDDEN_DIM)]

    d_W1 = [
        [d_h1p[j] * x_apres_attn[d] for d in range(EMBED_DIM)]
        for j in range(HIDDEN_DIM)
    ]
    d_b1 = list(d_h1p)
    for d in range(EMBED_DIM):
        d_xa[d] += sum(d_h1p[j] * W1[j][d] for j in range(HIDDEN_DIM))

    # === ETAPE 5 : connexion residuelle 1 ===
    d_attn_out = list(d_xa)
    d_hidden_last = list(d_xa)

    # === ETAPE 6 : backward de l'attention ===
    d_pw = [
        sum(d_attn_out[d] * valeurs[i][d] for d in range(EMBED_DIM)) for i in range(n)
    ]
    d_val = [
        [d_attn_out[d] * poids_attn[i] for d in range(EMBED_DIM)] for i in range(n)
    ]
    # Softmax backward
    d_sc = [0.0] * n
    for i in range(n):
        for j in range(n):
            if i == j:
                d_sc[i] += poids_attn[i] * (1 - poids_attn[i]) * d_pw[i]
            else:
                d_sc[i] -= poids_attn[j] * poids_attn[i] * d_pw[j]

    echelle = math.sqrt(EMBED_DIM)
    d_sc = [ds / echelle for ds in d_sc]

    d_q = [sum(d_sc[i] * cles[i][d] for i in range(n)) for d in range(EMBED_DIM)]
    d_cles = [[d_sc[i] * q[d] for d in range(EMBED_DIM)] for i in range(n)]

    d_Wq = [
        [d_q[r] * hidden[-1][c] for c in range(EMBED_DIM)] for r in range(EMBED_DIM)
    ]
    for d in range(EMBED_DIM):
        d_hidden_last[d] += sum(d_q[r] * Wq[r][d] for r in range(EMBED_DIM))

    d_Wk = [[0.0] * EMBED_DIM for _ in range(EMBED_DIM)]
    d_Wv = [[0.0] * EMBED_DIM for _ in range(EMBED_DIM)]
    d_hkv = [[0.0] * EMBED_DIM for _ in range(n)]
    for i in range(n):
        for r in range(EMBED_DIM):
            for c in range(EMBED_DIM):
                d_Wk[r][c] += d_cles[i][r] * hidden[i][c]
                d_Wv[r][c] += d_val[i][r] * hidden[i][c]
                d_hkv[i][c] += d_cles[i][r] * Wk[r][c]
                d_hkv[i][c] += d_val[i][r] * Wv[r][c]

    # === ETAPE 7 : gradient des embeddings ===
    d_tok_emb = [[0.0] * EMBED_DIM for _ in range(VOCAB_SIZE)]
    d_pos_emb = [[0.0] * EMBED_DIM for _ in range(n)]

    for i in range(n):
        d_h = list(d_hkv[i])
        if i == n - 1:
            for d in range(EMBED_DIM):
                d_h[d] += d_hidden_last[d]
        for d in range(EMBED_DIM):
            d_tok_emb[ids[i]][d] += d_h[d]
            d_pos_emb[i][d] += d_h[d]

    return {
        "d_W_out": d_W_out,
        "d_W2": d_W2,
        "d_b2": d_b2,
        "d_W1": d_W1,
        "d_b1": d_b1,
        "d_Wq": d_Wq,
        "d_Wk": d_Wk,
        "d_Wv": d_Wv,
        "d_tok_emb": d_tok_emb,
        "d_pos_emb": d_pos_emb,
    }


print("Fonctions forward et backward definies !")
print(
    "7 etapes de retropropagation : sortie -> W_out -> MLP -> attention -> embeddings"
)

In [ ]:
# ---
# ## Etape 4 : L'entrainement
#
# C'est la meme boucle que dans les lecons 2 et 3, mais avec le vrai LLM :
#
# 1. Prendre un Pokemon
# 2. Pour chaque position, predire la lettre suivante
# 3. Calculer l'erreur (la loss)
# 4. Calculer les gradients (backward)
# 5. Ajuster tous les poids un petit peu (SGD)
# 6. Recommencer
#
# > **Cette cellule va tourner pendant ~1-2 minutes.**
# > Pendant ce temps, lis les sections "En vrai..." plus bas !

NB_EPOCHS = 10  # <-- Change cette valeur ! Essaie 5 (rapide) ou 20 (meilleur)
vitesse = 0.01  # <-- Change cette valeur ! Essaie 0.005 (lent) ou 0.05 (rapide)

positions_par_mot = sum(len(p) + 1 for p in pokemons) / len(pokemons)
total_updates = int(NB_EPOCHS * len(pokemons) * positions_par_mot)

print(f"Entrainement : {NB_EPOCHS} epochs x {len(pokemons)} Pokemon")
print(f"  ~{total_updates:,} mises a jour des poids au total")
print()

_historique_loss_epochs = []

debut_chrono = time.time()

for epoch in range(NB_EPOCHS):
    random.shuffle(pokemons)
    loss_epoch = 0
    nb_epoch = 0

    for idx_mot, pokemon in enumerate(pokemons):
        mot = "." + pokemon + "."
        ids = [char_to_id[c] for c in mot]

        for i in range(1, len(ids)):
            seq = ids[:i][-CONTEXT:]
            cible = ids[i]

            # Forward
            probas, cache = forward_avec_cache(seq)
            loss_epoch += -math.log(probas[cible] + 1e-10)
            nb_epoch += 1

            # Backward
            grads = backward(cache, probas, cible)

            # SGD : ajuster chaque poids
            for v in range(VOCAB_SIZE):
                for d in range(EMBED_DIM):
                    W_out[v][d] -= vitesse * grads["d_W_out"][v][d]
            for j in range(HIDDEN_DIM):
                b1[j] -= vitesse * grads["d_b1"][j]
                for d in range(EMBED_DIM):
                    W1[j][d] -= vitesse * grads["d_W1"][j][d]
            for d in range(EMBED_DIM):
                b2[d] -= vitesse * grads["d_b2"][d]
                for j in range(HIDDEN_DIM):
                    W2[d][j] -= vitesse * grads["d_W2"][d][j]
            for r in range(EMBED_DIM):
                for c in range(EMBED_DIM):
                    Wq[r][c] -= vitesse * grads["d_Wq"][r][c]
                    Wk[r][c] -= vitesse * grads["d_Wk"][r][c]
                    Wv[r][c] -= vitesse * grads["d_Wv"][r][c]
            for tok_id in set(cache["ids"]):
                for d in range(EMBED_DIM):
                    tok_emb[tok_id][d] -= vitesse * grads["d_tok_emb"][tok_id][d]
            for pos in range(len(seq)):
                for d in range(EMBED_DIM):
                    pos_emb[pos % CONTEXT][d] -= vitesse * grads["d_pos_emb"][pos][d]

        if (idx_mot + 1) % 250 == 0:
            t = time.time() - debut_chrono
            print(
                f"  Epoch {epoch + 1}/{NB_EPOCHS} | Mot {idx_mot + 1:>5}/{len(pokemons)} | Loss : {loss_epoch / nb_epoch:.3f} | {t:.0f}s"
            )

    _historique_loss_epochs.append(loss_epoch / nb_epoch)

    t = time.time() - debut_chrono
    print(
        f"  === Epoch {epoch + 1} terminee | Loss : {loss_epoch / nb_epoch:.3f} | {t:.0f}s ==="
    )
    print()

duree = time.time() - debut_chrono
print(f"Entrainement termine en {duree:.0f} secondes !")
print(f"Loss : {loss_initiale:.3f} -> {loss_epoch / nb_epoch:.3f}")

# Visualisation de la courbe de loss
afficher_evolution_loss(_historique_loss_epochs, titre="Evolution de la loss par epoch")

---
### A toi de jouer ! (Exercice 2)

Observe le resultat de l'entrainement ci-dessus :
- Combien de temps a-t-il fallu par epoch ?
- La loss a-t-elle bien baisse ?

Si tu veux, re-execute la cellule d'entrainement en changeant
`NB_EPOCHS` (5 = rapide, 20 = meilleur) ou `vitesse` (0.005, 0.05).
Attention : il faudra re-executer les cellules d'initialisation aussi !

In [ ]:
# --- EXERCICE 2 : Observe les resultats, puis Shift + Entree ---

print("Resultats de l'entrainement :")
print(f"  Epochs : {NB_EPOCHS}")
print(f"  Vitesse : {vitesse}")
print(f"  Duree totale : {duree:.0f} secondes ({duree / NB_EPOCHS:.0f}s par epoch)")
print(f"  Loss initiale : {loss_initiale:.3f}")
print(f"  Loss finale : {loss_epoch / nb_epoch:.3f}")
print(f"  Amelioration : {(1 - (loss_epoch / nb_epoch) / loss_initiale) * 100:.0f}%")
print()
if loss_epoch / nb_epoch < 2.5:
    print("Le modele a bien appris !")
else:
    print("Le modele peut encore s'ameliorer. Essaie plus d'epochs !")

# Validation exercice 2
verifier(
    2,
    loss_epoch / nb_epoch < loss_initiale,
    f"Bien observe ! La loss est passee de {loss_initiale:.2f} a {loss_epoch / nb_epoch:.2f} en {NB_EPOCHS} epochs.",
    "Re-execute l'entrainement si la loss n'a pas baisse.",
)

---
## En vrai... pendant que le modele s'entraine

### Autograd vs notre backward manuel

On a ecrit ~60 lignes de code pour le backward pass. C'est beaucoup !

En vrai, les frameworks comme **PyTorch** font ca **automatiquement**.
Tu ecris juste le forward pass, et PyTorch calcule les gradients tout seul.
Ca s'appelle **l'autograd** (differentiation automatique).

C'est exactement ce que fait `microgpt.py` de Karpathy : il definit
des operations (`+`, `*`, `exp`) qui "se souviennent" comment elles ont
ete calculees, puis il remonte la chaine automatiquement.

### GPU vs CPU

Notre boucle Python fait les calculs **un par un**. Chaque multiplication,
chaque addition, une a la fois.

Un **GPU** (la carte graphique de ton PC) peut faire **des milliers de
multiplications en parallele**. C'est comme la difference entre :
- Un cuisinier qui prepare les plats un par un (CPU)
- Une brigade de 1000 cuisiniers qui preparent tous en meme temps (GPU)

C'est pour ca que l'entrainement de GPT-4 a utilise **25 000 GPU**
pendant **plusieurs mois**. Notre mini-LLM, avec ses ~2 800 parametres,
s'entraine en quelques minutes sur un seul CPU.

### Adam vs SGD

On utilise la descente de gradient la plus simple : **SGD** (Stochastic
Gradient Descent). A chaque pas, on corrige d'un montant fixe.

**Adam** est un optimiseur plus intelligent :
- Il **accelere** dans les zones plates (quand les gradients sont petits)
- Il **freine** dans les zones pentues (quand les gradients sont grands)
- Il se souvient des gradients precedents pour mieux orienter la correction

C'est comme un velo avec des vitesses : tu adaptes ton effort au terrain.
Presque tous les vrais LLM utilisent Adam (ou une variante comme AdamW).

---
## Ce qu'on n'a pas implémenté

Notre mini-LLM est **fonctionnel** mais simplifié. Voici ce que les vrais
LLM ajoutent :

| Technique | Notre mini-LLM | Les vrais LLM |
|-----------|----------------|---------------|
| **Batching** | 1 mot à la fois | 64-512 mots en parallèle |
| **LayerNorm** | Non | Oui (stabilise l'entraînement) |
| **Dropout** | Non | Oui (évite le sur-apprentissage) |
| **Multi-head** | 1 tête | 4-96 têtes en parallèle |
| **Multi-couches** | 1 couche | 6-96 couches empilées |
| **Optimizer** | SGD basique | Adam (plus intelligent) |
| **GPU** | Non (Python pur) | Oui (1000x plus rapide) |

Mais l'**algorithme** est le même ! La différence, c'est l'**échelle**.

In [ ]:
print("=== APRES entrainement ===")
print()
noms_apres = []
for _ in range(15):
    nom = generer(temperature=0.8)
    noms_apres.append(nom)
    print(f"  {nom.capitalize()}")

print()
print("--- Comparaison ---")
print()
print("AVANT (charabia) :")
for nom in noms_avant[:5]:
    print(f"  {nom.capitalize()}")
print()
print("APRES (ca ressemble a des Pokemon !) :")
for nom in noms_apres[:5]:
    print(f"  {nom.capitalize()}")

# Visualisation des predictions apres entrainement
_probas_post, _ = forward_avec_cache([char_to_id[c] for c in ".pik"][-CONTEXT:])
_top = sorted(range(VOCAB_SIZE), key=lambda i: -_probas_post[i])[:5]
afficher_barres(
    [_probas_post[i] for i in _top],
    [id_to_char[i] for i in _top],
    titre="Top 5 predictions apres '.pik' (apres entrainement)",
)

---
### A toi de jouer ! (Exercice 3)

Maintenant que le modele est entraine, change `ma_temperature` et
`mon_debut` pour explorer ce qu'il a appris.
- Temperature `0.1` : noms "sages" et repetitifs
- Temperature `2.0` : noms "fous" et originaux
- Debut `".pik"` : force le modele a continuer apres "pik"

In [ ]:
# --- EXERCICE 3 : Change la temperature et le debut, puis Shift + Entree ---
ma_temperature = 0.8  # <-- Essaie 0.1 (sage) ou 2.0 (fou) !
mon_debut = "."  # <-- Essaie ".pik", ".bul" ou ".dra" !

print(
    f"Generation APRES entrainement (temperature={ma_temperature}, debut='{mon_debut}') :"
)
print()
for _ in range(15):
    nom = generer(debut=mon_debut, temperature=ma_temperature)
    print(f"  {nom.capitalize()}")
print()
print("Compare avec l'exercice 1 : maintenant le modele sait ce qu'est un Pokemon !")

# Validation exercice 3
verifier(
    3,
    ma_temperature != 0.8 or mon_debut != ".",
    f"Genial ! Generation avec temperature={ma_temperature} et debut='{mon_debut}'.",
    "Change ma_temperature ou mon_debut pour explorer le modele entraine.",
)

---
## Ce qu'on a appris

```
Lecon 1 : Compter les lettres qui suivent        -> bigramme
Lecon 2 : Apprendre de ses erreurs               -> entrainement
Lecon 3 : Regarder plusieurs lettres en arriere   -> embeddings + contexte
Lecon 4 : Choisir les lettres importantes          -> attention
Lecon 5 : Assembler le tout                       -> mini-LLM
Lecon 6 : Entrainer pour de vrai                  -> retropropagation !
```

### Ce qu'on a fait dans cette lecon

1. **Charge ~1 000 noms de Pokemon**
2. **Implemente la retropropagation** : 7 etapes pour remonter les gradients
3. **Entraine le mini-LLM** avec la descente de gradient (SGD)
4. **Genere des noms de Pokemon inventes** qui ressemblent a de vrais Pokemon

### Ce qu'on a appris

- La **retropropagation** remonte l'erreur couche par couche
- L'**entrainement** = repeter forward + backward + mise a jour des poids
- Meme un modele de ~2 800 parametres peut **apprendre des patterns**
- La difference avec ChatGPT n'est pas l'algorithme, c'est **l'echelle**

---
*Tu as construit et entraine ton propre LLM. Felicitations !*

---

### Sources (ISO 42001)

- **Retropropagation et descente de gradient** : [microgpt.py](https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95) -- Andrej Karpathy, implementation complete du backward pass
- **Architecture GPT (embedding + attention + MLP)** : [Video "Let's build GPT"](https://www.youtube.com/watch?v=kCc8FmEb1nY) -- Andrej Karpathy (2023)
- **Cross-entropy loss et gradient softmax** : [3Blue1Brown - Neural Networks](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi) -- Grant Sanderson
- **"Attention Is All You Need"** : Vaswani et al., 2017, [arXiv:1706.03762](https://arxiv.org/abs/1706.03762)
- **Donnees d'entrainement** : [PokeAPI](https://pokeapi.co/) -- (c) Nintendo / Creatures Inc. / GAME FREAK inc., usage educatif uniquement